# Global stats scraper (crontabbed)

In [1]:
import time
import requests
from bs4 import BeautifulSoup

BASE_URL = 'https://adventofcode.com/2020/stats'
TIMESTAMP = int(time.time())

page_response = requests.get(BASE_URL)
page_content = BeautifulSoup(page_response.content, "html.parser")

stats_found = page_content.find('pre').find_all('a')
stats_found.reverse()
days_found = [stat.get_text().split()[0] for stat in stats_found]

to_write = ""

for day in range(1, 26):
    if str(day) in days_found:
        values = stats_found[day-1].get_text().split()
        txt_line = f"{TIMESTAMP},{day},{values[1]},{values[2]}\n"
    else:
        txt_line = f"{TIMESTAMP},{day},0,0\n"
    
    to_write += txt_line

with open("aoc_stats.csv", "a") as myfile:
    myfile.write(to_write)


# Leaderboard scraper (crontabbed)

In [2]:
import time
import requests
from bs4 import BeautifulSoup

BASE_URL = 'https://adventofcode.com/2020/leaderboard'
TIMESTAMP = int(time.time())

page_response = requests.get(BASE_URL)
page_content = BeautifulSoup(page_response.content, "html.parser")

stats_found = page_content.find_all(class_ = 'leaderboard-entry')

to_write = ""

for stat_found in stats_found:
    score = int(stat_found.find(class_ = 'leaderboard-totalscore').get_text())
    
    if stat_found.find(class_ = 'leaderboard-anon'):
        name = stat_found.find(class_ = 'leaderboard-anon').get_text()
    
    else:
        if not stat_found.find(class_ = 'leaderboard-position'):
            name_location = stat_found.contents[3:5]
        else:
            name_location = stat_found.contents[4:6]
        
        if name_location[0].name == 'a':
            name = name_location[0].get_text()
        elif name_location[0].name == 'span':
            name = name_location[1]
        else:
            name = 'THIS SHOULD NOT HAPPEN. SORRY.'
    
    to_write += f"{TIMESTAMP},{name},{score}\n"

with open("aoc_leaderboard.csv", "a") as myfile:
    myfile.write(to_write)


# Day 13 (Clean)

In [41]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE')

response = requests.get(f'https://adventofcode.com/2020/day/13/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['1003681',
 '23,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,37,x,x,x,x,x,431,x,x,x,x,x,x,x,x,x,x,x,x,13,17,x,x,x,x,19,x,x,x,x,x,x,x,x,x,x,x,409,x,x,x,x,x,x,x,x,x,41,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,x,29']

# DAY 14

In [120]:
with open('input.txt') as f:
    input_ = f.read().splitlines()

input_

['mask = 11110100010101111011001X0100XX00100X',
 'mem[17610] = 1035852',
 'mem[55284] = 229776690',
 'mem[16166] = 12685380',
 'mem[8340] = 16011',
 'mask = 0X1X0X010101011X10X101000X0001110100',
 'mem[968] = 15992',
 'mem[32758] = 7076',
 'mem[30704] = 1701',
 'mem[33719] = 58012',
 'mem[20818] = 25927237',
 'mem[16718] = 46485',
 'mask = 111001111X0X0X101X111X1X001XX0011010',
 'mem[2115] = 14848',
 'mem[42753] = 617',
 'mem[56076] = 9933868',
 'mem[19106] = 43503',
 'mem[10073] = 32909',
 'mem[40830] = 1959686',
 'mask = X11X00000XX1011X10000X01110000X0001X',
 'mem[41605] = 13245557',
 'mem[6571] = 7973763',
 'mem[46231] = 28527162',
 'mem[44901] = 163334644',
 'mask = 0101XXX1X10XX1101011110000000010010X',
 'mem[53492] = 357272',
 'mem[32816] = 35015',
 'mem[6965] = 11280352',
 'mem[27745] = 160101',
 'mem[26728] = 1260',
 'mask = 1XXX0100XX1X0X10101100011101101X0111',
 'mem[22010] = 28123044',
 'mem[42154] = 82539',
 'mem[54914] = 22078',
 'mem[7185] = 436',
 'mem[58583] = 25334197

In [121]:
part_one_values = {}

for idx, elem in enumerate(input_):
    if elem[:4] == 'mask':
        current_mask = elem.split(' = ')[1]
        continue
    
    else:
        mem_address = elem.split('] = ')[0][4:]
        mem_value = list(f"{int(elem.split('] = ')[1]):>036b}")
        
        for idx, char in enumerate(mem_value):
            if current_mask[idx] == 'X':
                continue
            else:
                mem_value[idx] = current_mask[idx]
        
        part_one_values[mem_address] = int(''.join(mem_value), 2)

print(sum(part_one_values.values()))

10885823581193


In [122]:
part_two_values = {}

for idx, elem in enumerate(input_):
    if elem[:4] == 'mask':
        current_mask = elem.split(' = ')[1]
        continue
    
    else:
        mem_address = list(f"{int(elem.split('] = ')[0][4:]):>036b}") 
        mem_value = int(elem.split('] = ')[1])
        
        for idx, char in enumerate(mem_address):
            if current_mask[idx] == '0':
                continue
            else:
                mem_address[idx] = current_mask[idx]
        
        print(''.join(mem_address), mem_value)
        all_addresses = get_addresses(''.join(mem_address))
        for address in all_addresses:
            part_two_values[str(address)] = mem_value
        print(all_addresses)
#     break

print(sum(part_two_values.values()))

11110100010101111011011X0100XX00101X 1035852
[65590223882, 65590223883, 65590223946, 65590223947, 65590224010, 65590224011, 65590224074, 65590224075, 65590227978, 65590227979, 65590228042, 65590228043, 65590228106, 65590228107, 65590228170, 65590228171]
11110100010101111011111X0111XX11110X 229776690
[65590257468, 65590257469, 65590257532, 65590257533, 65590257596, 65590257597, 65590257660, 65590257661, 65590261564, 65590261565, 65590261628, 65590261629, 65590261692, 65590261693, 65590261756, 65590261757]
11110100010101111011001X1111XX10111X 12685380
[65590210350, 65590210351, 65590210414, 65590210415, 65590210478, 65590210479, 65590210542, 65590210543, 65590214446, 65590214447, 65590214510, 65590214511, 65590214574, 65590214575, 65590214638, 65590214639]
11110100010101111011001X0100XX01110X 16011
[65590207516, 65590207517, 65590207580, 65590207581, 65590207644, 65590207645, 65590207708, 65590207709, 65590211612, 65590211613, 65590211676, 65590211677, 65590211740, 65590211741, 655902118

0110011111X100101X11111XX111X0001X0X 51920318
[27869832968, 27869832969, 27869832972, 27869832973, 27869833096, 27869833097, 27869833100, 27869833101, 27869835016, 27869835017, 27869835020, 27869835021, 27869835144, 27869835145, 27869835148, 27869835149, 27869837064, 27869837065, 27869837068, 27869837069, 27869837192, 27869837193, 27869837196, 27869837197, 27869839112, 27869839113, 27869839116, 27869839117, 27869839240, 27869839241, 27869839244, 27869839245, 27870095112, 27870095113, 27870095116, 27870095117, 27870095240, 27870095241, 27870095244, 27870095245, 27870097160, 27870097161, 27870097164, 27870097165, 27870097288, 27870097289, 27870097292, 27870097293, 27870099208, 27870099209, 27870099212, 27870099213, 27870099336, 27870099337, 27870099340, 27870099341, 27870101256, 27870101257, 27870101260, 27870101261, 27870101384, 27870101385, 27870101388, 27870101389, 27903387400, 27903387401, 27903387404, 27903387405, 27903387528, 27903387529, 27903387532, 27903387533, 27903389448, 2790

In [112]:
from itertools import product

def get_addresses(mask):
    listed_mask = list(mask)
#     print(listed_mask)
    X_indices = [i for i, x in enumerate(listed_mask) if x == "X"]
#     print(X_indices)
    repl_list = list(product(['0', '1'], repeat=len(X_indices)))
#     print(repl_list)
    
    addr_list = []
    for repl in repl_list:
        for repl_idx, repl_value in enumerate(repl):
            listed_mask[X_indices[repl_idx]] = repl_value
#         print(''.join(listed_mask))
        addr_list.append(int(''.join(listed_mask), 2))
#             print(repl_value)
#         for idx in X_indices:
#             mask[idx]print(elem)

    return addr_list

get_addresses('11110100010101111011011X0100XX00101X')

[65590223882,
 65590223883,
 65590223946,
 65590223947,
 65590224010,
 65590224011,
 65590224074,
 65590224075,
 65590227978,
 65590227979,
 65590228042,
 65590228043,
 65590228106,
 65590228107,
 65590228170,
 65590228171]

# Day 15

In [39]:
with open('input.txt') as f:
    input_ = f.read().splitlines()

input_ = [int(i) for i in input_[0].split(',')]
input_

# 13,16,0,12,15,1

[9, 19, 1, 6, 0, 5, 4]

In [41]:
while len(input_) < 30000000:
    
    indices = [idx for idx, value in enumerate(input_) if value == input_[-1]]
    if len(indices) == 1:
        new_num = 0
    else:
        new_num = indices[-1] - indices[-2]
    
#     print(indices, new_num)
#     new_num = input_.count(input_[-1]) - 1
    input_.append(new_num)
#     print(input_)

print(new_num)

counters = { value: {'times': 0, 'indices': [idx]} for idx, value in enumerate(input_)}
current_value = input_[-1]
current_idx = len(input_)

print(counters)
print(current_value, current_idx)

while current_idx < 30000000:
    
    if len(counters[current_value]['indices']) == 1:
        new_value = 0
    else:
        new_value = counters[current_value]['indices'][1] - counters[current_value]['indices'][0]
    
#     print(new_value)
    if new_value in counters.keys():
        counters[new_value]['times'] += 1
        counters[new_value]['indices'].append(current_idx)
        if len(counters[new_value]['indices']) > 2:
            counters[new_value]['indices'].pop(0)
    else:
        counters[new_value] = {
            'times': 0,
            'indices': [current_idx]
        }
    
    current_idx += 1
    current_value = new_value

print(current_value)


KeyboardInterrupt: 

# Day 16

In [57]:
with open('input.txt') as f:
    input_ = f.read().splitlines()

input_

['departure location: 27-374 or 395-974',
 'departure station: 40-287 or 295-953',
 'departure platform: 27-554 or 570-961',
 'departure track: 40-604 or 618-958',
 'departure date: 43-842 or 850-972',
 'departure time: 30-302 or 315-952',
 'arrival location: 32-478 or 496-950',
 'arrival station: 48-733 or 755-969',
 'arrival platform: 37-260 or 276-954',
 'arrival track: 40-512 or 519-964',
 'class: 34-277 or 284-966',
 'duration: 25-648 or 672-961',
 'price: 28-684 or 705-956',
 'route: 30-157 or 176-950',
 'row: 47-881 or 903-970',
 'seat: 38-705 or 727-959',
 'train: 40-195 or 217-961',
 'type: 28-858 or 879-958',
 'wagon: 31-543 or 554-967',
 'zone: 49-790 or 816-953',
 '',
 'your ticket:',
 '103,79,61,97,109,67,89,83,59,53,139,131,101,113,149,127,71,73,107,137',
 '',
 'nearby tickets:',
 '473,926,599,474,412,65,885,833,533,780,539,222,177,762,132,583,414,450,177,113',
 '110,74,420,522,243,130,575,115,553,92,157,193,370,949,334,74,53,462,837,822',
 '769,341,505,146,841,238,53,8,3

In [58]:
fields = input_[:20]
my_ticket = [int(i) for i in input_[22].split(',')]
other_tickets = [[int(i) for i in line.split(',')] for line in input_[25:]]

fields_values = {}
for field in fields:
    name = field.split(': ')[0]
    allowed_ranges = field.split(': ')[1].split(' or ')
    
    range_one = [int(j) for j in allowed_ranges[0].split('-')]
    list_one = [k for k in range(range_one[0], range_one[1]+1)]
    
    range_two = [int(j) for j in allowed_ranges[1].split('-')]
    list_two = [k for k in range(range_two[0], range_two[1]+1)]
    
    fields_values[name] = list_one + list_two

values_allowed = sorted(list(set([item for sublist in [value for _, value in fields_values.items()] for item in sublist])))
# print(values_allowed)

flat_other_tickets = [item for sublist in other_tickets for item in sublist]
scanning_error = 0
good_tickets = []
for ticket in other_tickets:
    for idx, number in enumerate(ticket):
        if number not in values_allowed:
            scanning_error += number
            break
        
        if idx == len(ticket) - 1:
            good_tickets.append(ticket)

print(scanning_error)
print(len(good_tickets))

unknown_fields = list(map(list, zip(*good_tickets)))
named_fields = []
for idx, unknown_field in enumerate(unknown_fields):
    possible_fields = []
    
    for name, values in fields_values.items():
        if len(set(unknown_field) - set(values)) == 0:
            possible_fields.append(name)
    
    named_fields.append(possible_fields)

real_named_fields = []
indexes_to_check = [i for i in range(len(named_fields))]
print(len(named_fields))

while indexes_to_check != []:
    print(indexes_to_check)
    
    for index in indexes_to_check:
        list_to_check = named_fields[index]
        
        if len(list_to_check) == 1:
            name = list_to_check[0]
            real_named_fields.append([index, name])
            indexes_to_check.remove(index)
            
            for n in named_fields:
                if name in n:
                    n.remove(name)

print(real_named_fields)
departure_fields = [val for val in real_named_fields if 'departure' in val[1]]

result = 1

for f in departure_fields:
    result *= my_ticket[f[0]]

print(result)

26941
190
20
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19]
[1, 2, 3, 4, 5, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 19]
[1, 2, 3, 4, 5, 7, 9, 11, 13, 14, 15, 16, 17, 18, 19]
[1, 2, 4, 5, 7, 9, 11, 13, 14, 15, 16, 17, 18, 19]
[2, 4, 5, 7, 9, 11, 14, 15, 16, 17, 18, 19]
[2, 4, 7, 9, 11, 14, 15, 16, 17, 18, 19]
[4, 7, 9, 11, 14, 16, 17, 18]
[4, 7, 11, 14, 16, 17, 18]
[4, 11, 16, 17, 18]
[11, 16, 17]
[11, 16]
[11]
[[6, 'arrival station'], [0, 'zone'], [12, 'row'], [10, 'route'], [8, 'price'], [3, 'train'], [1, 'departure time'], [13, 'departure location'], [5, 'departure track'], [2, 'departure platform'], [15, 'departure date'], [19, 'departure station'], [9, 'wagon'], [7, 'seat'], [14, 'class'], [4, 'arrival track'], [18, 'duration'], [17, 'arrival platform'], [16, 'arrival location'], [11, 'type']]
634796407951


# Day 17

In [186]:
from itertools import product
from copy import deepcopy

def surround_with(list_3d, placeholder='.'):
    result = deepcopy(list_3d)
    
    X = len(result[0])
    Y = len(result[0][0])
    Z = len(result)
    
    y_layer = [placeholder for _ in range(Y+2)]
    z_layer = [deepcopy(y_layer) for _ in range(X+2)]

    for z in result:        
        for x in z:
            x.insert(0, placeholder)
            x.append(placeholder)
        
        z.insert(0, deepcopy(y_layer))
        z.append(deepcopy(y_layer))
    
    result.insert(0, deepcopy(z_layer))
    result.append(deepcopy(z_layer))
    
    return result

def surround_with_4d(list_4d, placeholder='.'):
    result = deepcopy(list_4d)
    
    X = len(result[0][0])
    Y = len(result[0][0][0])
    Z = len(result[0])
    W = len(result)
    
    y_layer = [placeholder for _ in range(Y+2)]
    z_layer = [deepcopy(y_layer) for _ in range(X+2)]
    w_layer = [deepcopy(z_layer) for _ in range(Z+2)]
    
    for w in result:
        for z in w:        
            for x in z:
                x.insert(0, placeholder)
                x.append(placeholder)

            z.insert(0, deepcopy(y_layer))
            z.append(deepcopy(y_layer))

        w.insert(0, deepcopy(z_layer))
        w.append(deepcopy(z_layer))
    
    result.insert(0, deepcopy(w_layer))
    result.append(deepcopy(w_layer))
    
    return result

def star_neighbors_3d(list_3d, x, y, z):    
    X = len(list_3d[0])
    Y = len(list_3d[0][0])
    Z = len(list_3d)
    
    neighbors = list(product([-1, 0, 1], repeat=3))
    neighbors.remove((0, 0, 0))
    
    real_neighbors = []
    for neighbor in neighbors:
        new_x = x+neighbor[0]
        new_y = y+neighbor[1]
        new_z = z+neighbor[2]
        
        if new_x % X == new_x and new_y % Y == new_y and  new_z % Z == new_z:
            real_neighbors.append(list_3d[new_z][new_x][new_y])
            
    return real_neighbors


def star_neighbors_4d(list_4d, x, y, z, w):
    X = len(list_4d[0][0])
    Y = len(list_4d[0][0][0])
    Z = len(list_4d[0])
    W = len(list_4d)
    
    neighbors = list(product([-1, 0, 1], repeat=4))
    neighbors.remove((0, 0, 0, 0))
    
    real_neighbors = []
    for neighbor in neighbors:
        new_x = x+neighbor[0]
        new_y = y+neighbor[1]
        new_z = z+neighbor[2]
        new_w = w+neighbor[3]
        
        if new_x % X == new_x and new_y % Y == new_y and  new_z % Z == new_z and new_w % W == new_w:
            real_neighbors.append(list_4d[new_w][new_z][new_x][new_y])
            
    return real_neighbors


def count_nested(list_, value):
    count = 0
    
    for elem in list_:
        if type(elem) == list:
            count += count_nested(elem, value)
        elif elem == value:
            count +=1
    
    return count




In [188]:
example = [
    [
        [1, 2, 3, 4],
        [5, 6, 7, 8],
        [9, 10, 11, 12]
    ],
    [
        [13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 1, 24]
        
    ],
    [
        [25, 26, 27, 28],
        [29, 30, 31, 32],
        [33, 1, 35, 36]
    ]
]

count_nested(example, 1)
# toto = surround_with(example, '.')
# surround_with(surround_with(toto, '_'))


3

In [203]:
with open('input.txt') as f:
    input_ = f.read().splitlines()

input_ = [ [list(s) for s in input_] ]
input_4d = [ input_ ]

# # Part 1
# output_ = deepcopy(input_)

# for _ in range(6):
#     input_ = deepcopy(surround_with(output_))
#     output_ = deepcopy(input_)

#     for z in range(len(input_)):
#         for x in range(len(input_[0])):
#             for y in range(len(input_[0][0])):
#                 myself = input_[z][x][y]
#                 my_neighbors = star_neighbors_3d(input_, x, y, z)

#                 if (myself == '#' and my_neighbors.count('#') in [2, 3]) or (myself == '.' and my_neighbors.count('#') == 3):
#                     output_[z][x][y] = '#'
#                 else:
#                     output_[z][x][y] = '.'

# print(count_nested(output_, '#'))

# Part 2
output_4d = deepcopy(input_4d)
# print(surround_with_4d(output_4d))

for _ in range(12):
    input_4d = deepcopy(surround_with_4d(output_4d))
    output_4d = deepcopy(input_4d)
    
    for w in range(len(input_4d)):
        for z in range(len(input_4d[0])):
            for x in range(len(input_4d[0][0])):
                for y in range(len(input_4d[0][0][0])):
                    myself = input_4d[w][z][x][y]
                    my_neighbors = star_neighbors_4d(input_4d, x, y, z, w)

                    if (myself == '#' and my_neighbors.count('#') in [2, 3]) or (myself == '.' and my_neighbors.count('#') == 3):
                        output_4d[w][z][x][y] = '#'
                    else:
                        output_4d[w][z][x][y] = '.'

print(count_nested(output_4d, '#'))


23144


# Day 17 > speed run

In [40]:
# %%timeit

from itertools import product


def initialize_coordinates(list_, dimension):
    coordinates = set()

    for x, line in enumerate(input_):
        for y, cube in enumerate(line):
            if cube == '#':
                zeroes = (0,) * (dimension-2)
                coordinates.add((x, y) + zeroes)

    return coordinates


def floating_neighbors(point):
    dimension = len(point)
    
    neighbors_space = list(product([-1, 0, 1], repeat=dimension))
    neighbors_space.remove((0,) * dimension)
    
    return list(tuple(a + b for a, b in zip(point, neighbor)) for neighbor in neighbors_space)


# ----- Début du script ----- #


with open('input.txt') as f:
    input_ = f.read().splitlines()

input_ = [list(s) for s in input_]

DIMENSION = 4

active_cubes = initialize_coordinates(input_, DIMENSION)

for _ in range(6):

    new_active_cubes = set()
    close_neighbors = {}

    for active_cube in active_cubes:
        active_neighbor_counter = 0
        
        for neighbor in floating_neighbors(active_cube):
            if neighbor in active_cubes:
                active_neighbor_counter += 1
            
            if neighbor in close_neighbors.keys():
                close_neighbors[neighbor] += 1
            else:
                close_neighbors[neighbor] = 1
        
        if active_neighbor_counter in [2, 3]:
            new_active_cubes.add(active_cube)
    
    for cube, active_neighbors in close_neighbors.items():
        if active_neighbors == 3:
            new_active_cubes.add(cube)
    
    active_cubes = new_active_cubes

len(new_active_cubes)

1908

# Day 18

In [151]:
with open('input.txt') as f:
    input_ = f.read().splitlines()

input_

['(4 * (3 * 2 + 2) * (9 * 7 * 5 * 4 * 9) * (7 * 7 + 7 * 4 + 9)) + 6 * 4 + 8 + ((6 * 5) * 4 * (2 * 8 + 4 + 7 * 9 + 3) * 2 + 6) + 3',
 '(3 * 2 + (6 + 4 + 3 * 6 * 4 * 8) * 4 + 4) + 2',
 '(9 * 8 * 4 * 8 * (4 + 8 + 7 + 7 * 5) + 3) * 5',
 '((4 + 8 * 3) * (6 * 9)) + (3 + 9 + (9 + 4 * 4 + 8 * 7)) * 9 * (2 * (9 + 9 + 3 + 9) + 5 + 6 + 2 + 6) + 6 * 4',
 '4 * 2 + ((7 * 3 * 7) * 6) + ((9 + 3) + 2 + 7 + 8) * (4 * 9 * (8 * 5) + (3 * 2 + 2) + 3)',
 '3 + 6 + 3 + 9',
 '9 + 6 * 4 * 2 + 7 * 2',
 '(2 * 2) + (9 * 6 * 6 * (8 + 7)) + 3 * 8',
 '(9 * 3 + (5 * 4 + 7 * 9) + 2 + 7 * 9) + (4 * 9 * 3 + 8 + (4 * 8 + 5 + 8) + 8) * 3 + 6 + ((6 * 6) * 5 + 7 + 3 * 9 * 6) + 2',
 '(9 + 5) + 7 * 9 * 7',
 '4 * 6 + 3 + (3 * 7 * 4) * (5 * 9 * (4 + 6 + 9) + 9) + 4',
 '5 * (4 * 9 * 2 * 5 + (7 + 5) * 6) + (7 + 5 * 2 * (8 + 8 * 7 * 2 * 3 * 6) + 9 * 8) + 9 + 5',
 '2 + (9 * 3 + 5 * (6 * 6 * 3 + 5) * 7) + ((3 + 8 + 2) + 4 + 9) * 7 + 6 * 2',
 '3 * 6 + 8 + 6 + (5 * 8 + 8)',
 '4 * 7 + 9 + 4 + 2 * (7 * (8 * 4 * 5 + 5 * 7 * 2))',
 '(2 * 7

In [152]:
import numpy 

def operate(val_one, operator, val_two):
    if operator == '+':
        return val_one + val_two
    if operator == '*':
        return val_one * val_two

def compute(string):
#     print(string)
    if string[0] == '(':
        string = string[1:]
    if string[-1] == ')':
        string = string[:-1]
    
    splitted = string.split(' ')
    
    if len(splitted) > 3:
        first_val = operate(int(splitted[0]), splitted[1], int(splitted[2]))
        new_calc = str(first_val) + ' ' + ' '.join(splitted[3:])
        return compute(new_calc)
    
    else:
        return operate(int(splitted[0]), splitted[1], int(splitted[2]))

def compute_v2(string):
    if string[0] == '(':
        string = string[1:]
    if string[-1] == ')':
        string = string[:-1]
    
    splitted = string.split(' ')
    
    print(splitted)
    
    if '+' in splitted:
        if '*' not in splitted:
            return sum([int(i) for i in splitted[::2]])
        
        if len(splitted) > 3:
            plus_idx = splitted.index('+')
            first_val = operate(int(splitted[plus_idx-1]), splitted[plus_idx], int(splitted[plus_idx+1]))
            new_calc = (' '.join(splitted[:plus_idx-1]) + ' ' + str(first_val) + ' ' + ' '.join(splitted[plus_idx+2:])).strip()
#             print(new_calc)
            return compute_v2(new_calc)
        else:
#             print(operate(int(splitted[0]), splitted[1], int(splitted[2])))
            return operate(int(splitted[0]), splitted[1], int(splitted[2]))
            
    else:
#         pass
        print(splitted)
        values = [int(i) for i in splitted[::2]]
        print(values)
        return numpy.prod(values)

def calc(string):
    nested_calcs = re.findall('(\([^\(\)]*\))', string)
    
    while nested_calcs != []:
        for calculation in nested_calcs:
#             print(string)
            string = string.replace(calculation, str(compute(calculation)))

        nested_calcs = re.findall('(\([^\(\)]*\))', string)
    
    return compute(string)

def calc_v2(string):
    nested_calcs = re.findall('(\([^\(\)]*\))', string)
    
    while nested_calcs != []:
        for calculation in nested_calcs:
#             print(string)
            string = string.replace(calculation, str(compute_v2(calculation)))

        nested_calcs = re.findall('(\([^\(\)]*\))', string)
    
    return compute_v2(string)


In [146]:
s = ['2', '*', '3']
[int(i) for i in s[::2]]

[2, 3]

In [154]:
somme = 0
for calcul in input_:
#     print(calcul)
#     print(calc_v2(calcul))
    somme += calc_v2(calcul)

somme



# somme = 0
# for calcul in input_:
#     print(calc(calcul))
#     somme += calc(calcul)

# somme

['3', '*', '2', '+', '2']
['3', '*', '4']
['3', '*', '4']
[3, 4]
['9', '*', '7', '*', '5', '*', '4', '*', '9']
['9', '*', '7', '*', '5', '*', '4', '*', '9']
[9, 7, 5, 4, 9]
['7', '*', '7', '+', '7', '*', '4', '+', '9']
['7', '*', '14', '*', '4', '+', '9']
['7', '*', '14', '*', '13']
['7', '*', '14', '*', '13']
[7, 14, 13]
['6', '*', '5']
['6', '*', '5']
[6, 5]
['2', '*', '8', '+', '4', '+', '7', '*', '9', '+', '3']
['2', '*', '12', '+', '7', '*', '9', '+', '3']
['2', '*', '19', '*', '9', '+', '3']
['2', '*', '19', '*', '12']
['2', '*', '19', '*', '12']
[2, 19, 12]
['4', '*', '12', '*', '11340', '*', '1274']
['4', '*', '12', '*', '11340', '*', '1274']
[4, 12, 11340, 1274]
['30', '*', '4', '*', '456', '*', '2', '+', '6']
['30', '*', '4', '*', '456', '*', '8']
['30', '*', '4', '*', '456', '*', '8']
[30, 4, 456, 8]
['693463680', '+', '6', '*', '4', '+', '8', '+', '437760', '+', '3']
['693463686', '*', '4', '+', '8', '+', '437760', '+', '3']
['693463686', '*', '12', '+', '437760', '+', '3']

[5, 112756, 9, 11]
['4', '*', '4', '*', '5', '+', '9']
['4', '*', '4', '*', '14']
['4', '*', '4', '*', '14']
[4, 4, 14]
['7', '+', '6', '*', '2', '*', '9', '+', '3']
['13', '*', '2', '*', '9', '+', '3']
['13', '*', '2', '*', '12']
['13', '*', '2', '*', '12']
[13, 2, 12]
['5', '+', '3', '*', '3', '+', '8', '+', '5']
['8', '*', '3', '+', '8', '+', '5']
['8', '*', '11', '+', '5']
['8', '*', '16']
['8', '*', '16']
[8, 16]
['224', '*', '312']
['224', '*', '312']
[224, 312]
['69888', '+', '5', '*', '128', '*', '8', '+', '2']
['69893', '*', '128', '*', '8', '+', '2']
['69893', '*', '128', '*', '10']
['69893', '*', '128', '*', '10']
[69893, 128, 10]
['7', '+', '9', '+', '4']
['4', '+', '4', '*', '20']
['8', '*', '20']
['8', '*', '20']
[8, 20]
['6', '+', '5', '+', '4', '+', '8', '+', '9']
['3', '+', '5', '+', '32', '+', '5']
['45', '+', '4']
['5', '+', '9', '+', '4']
['5', '*', '6', '*', '18', '+', '8', '*', '7', '+', '9']
['5', '*', '6', '*', '26', '*', '7', '+', '9']
['5', '*', '6', '*', '26'

['12', '*', '4', '*', '340205', '+', '67405']
['12', '*', '4', '*', '407610']
['12', '*', '4', '*', '407610']
[12, 4, 407610]
['4', '+', '4', '+', '4', '*', '9', '*', '7', '+', '6']
['8', '+', '4', '*', '9', '*', '7', '+', '6']
['12', '*', '9', '*', '7', '+', '6']
['12', '*', '9', '*', '13']
['12', '*', '9', '*', '13']
[12, 9, 13]
['9', '+', '5', '+', '1404', '+', '9']
['6', '+', '4', '+', '1427', '*', '8', '*', '4']
['10', '+', '1427', '*', '8', '*', '4']
['1437', '*', '8', '*', '4']
['1437', '*', '8', '*', '4']
[1437, 8, 4]
['3', '*', '2']
['3', '*', '2']
[3, 2]
['2', '*', '4', '*', '6', '+', '2', '*', '5', '*', '4']
['2', '*', '4', '*', '8', '*', '5', '*', '4']
['2', '*', '4', '*', '8', '*', '5', '*', '4']
[2, 4, 8, 5, 4]
['3', '*', '7', '*', '3', '*', '4', '+', '6']
['3', '*', '7', '*', '3', '*', '10']
['3', '*', '7', '*', '3', '*', '10']
[3, 7, 3, 10]
['2', '+', '1280']
['5', '*', '5', '*', '630', '*', '1282']
['5', '*', '5', '*', '630', '*', '1282']
[5, 5, 630, 1282]
['2', '+', '

461295257566346

In [96]:
toto = [1, 2, 3, 4, 5, 6, 7, 8, 9]

toto[:2] + toto[5:]

[1, 2, 6, 7, 8, 9]

In [60]:
test = input_[2] # '4 * 6' # 
test

'((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2'

In [45]:
import re

s = test
result = re.findall('(\([^\(\)]*\))', s)
print(result)
test[:-1]

['(2 + 4 * 9)', '(6 + 9 * 8 + 6)']


'((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * '

In [74]:
calc(test)

((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2
(54 * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2
(54 * 126 + 6) + 2 + 4 * 2


13632

In [27]:
if '(' in test:
    to_calc = re.search('\((.*)\)', s)
    print(to_calc.group(1))

(2 + 4 * 9) * (6 + 9 * 8 + 6) + 6


In [41]:
def calculate(string):
    if '(' in string:
        to_calc = re.findall('\(([^(\).)]*)\)', string)
        print(to_calc)
        for substring in to_calc:
            calculate(substring)
    else:
        return 5

calculate(test)

['2 + 4 * 9', '6 + 9 * 8 + 6']


# Day 19

In [1]:
import os 
import requests
from dotenv import load_dotenv

load_dotenv()

SESSION_COOKIE = os.getenv('SESSION_COOKIE')
response = requests.get(f'https://adventofcode.com/2020/day/19/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['132: 80 20 | 46 54',
 '107: 58 20 | 3 54',
 '55: 126 20 | 107 54',
 '93: 110 29',
 '8: 42',
 '75: 92 20 | 103 54',
 '113: 91 20 | 35 54',
 '94: 54 64 | 20 21',
 '46: 54 20 | 110 54',
 '26: 54 74 | 20 94',
 '61: 20 76 | 54 25',
 '128: 38 20 | 29 54',
 '129: 20 81 | 54 49',
 '21: 20 49 | 54 9',
 '103: 54 2 | 20 51',
 '9: 54 54 | 20 20',
 '82: 54 9 | 20 29',
 '127: 20 54 | 54 54',
 '59: 40 54 | 87 20',
 '0: 8 11',
 '105: 20 125 | 54 1',
 '78: 82 54 | 132 20',
 '88: 20 63 | 54 13',
 '53: 20 46 | 54 38',
 '62: 54 123 | 20 65',
 '116: 20 19 | 54 66',
 '32: 118 20 | 26 54',
 '81: 54 54',
 '45: 54 50 | 20 98',
 '15: 124 54 | 73 20',
 '118: 59 54 | 85 20',
 '64: 49 20 | 13 54',
 '10: 20 95 | 54 128',
 '95: 54 80 | 20 81',
 '11: 42 31',
 '97: 20 46',
 '108: 20 77 | 54 88',
 '63: 110 110',
 '125: 29 54 | 12 20',
 '79: 20 29 | 54 49',
 '43: 54 104 | 20 6',
 '66: 115 54 | 112 20',
 '67: 110 80',
 '44: 54 12 | 20 63',
 '37: 99 54 | 63 20',
 '74: 120 54 | 48 20',
 '27: 54 41 | 20 105',
 '14: 20 101

In [8]:
def parse_rule(s):

    this_rule, elements = s.split(": ")
    if '"' not in elements:
        elements = [[int(rule) for rule in choice.split()] for choice in elements.split("|")]

    return (int(this_rule), elements)

def try_match(message, tested_rules, all_rules):
    '''
    Given a message and a list of rules, is there a way to reproduce the message using the rules?
    Returns True or False
    '''
    
    # In case the whole message OR all the rules have been tested... 
    if message == '' or tested_rules == []:
        # It's a match if no rule remains and the whole message was consumed
        return message == '' and tested_rules == []
    
    r = all_rules[tested_rules[0]]
    
    # If the rule is a char ("a" or "b")...
    if '"' in r:
        # ...and if the first char of the message is this char...
        if message[0] in r:
            # ...try_match the next char with the next rule
            return try_match(message[1:], tested_rules[1:], all_rules)
        else:
            # The first char is wrong
            return False
    else:
        # try_match with new (expanded) rules
        return any(try_match(message, t + tested_rules[1:], all_rules) for t in r)


def result(input_):
    raw_rules = input_[:input_.index('')]
    dict_rules = dict(parse_rule(rule) for rule in raw_rules)
    messages = input_[input_.index('') + 1:]
    
    part_one = sum(try_match(m, [0], dict_rules) for m in messages)
    
    raw_rules += ["8: 42 | 42 8", "11: 42 31 | 42 11 31"]
    dict_rules = dict(parse_rule(rule) for rule in raw_rules)
    
    part_two = sum(try_match(m, [0], dict_rules) for m in messages)
    
    return part_one, part_two


result(input_)


(118, 246)

In [55]:
input_ = '''0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b"

ababbb
bababa
abbbab
aaabbb
aaaabbb
'''.splitlines()

input_

['0: 4 1 5',
 '1: 2 3 | 3 2',
 '2: 4 4 | 5 5',
 '3: 4 5 | 5 4',
 '4: "a"',
 '5: "b"',
 '',
 'ababbb',
 'bababa',
 'abbbab',
 'aaabbb',
 'aaaabbb']

In [57]:
rules = {val.split(': ')[0]: [x.split(' ') for x in val.split(': ')[1].replace('"', '').split(' | ')] for val in input_[:input_.index('')]}
rules


{'0': [['4', '1', '5']],
 '1': [['2', '3'], ['3', '2']],
 '2': [['4', '4'], ['5', '5']],
 '3': [['4', '5'], ['5', '4']],
 '4': [['a']],
 '5': [['b']]}

In [109]:
from copy import deepcopy
import re

def transform(current_rule, all_rules):
    new_rules = [current_rule]
    for rule in current_rule:
        if rule in ['a', 'b']:
            continue
        
        this_rule = all_rules[rule]
        this_rule_idx = current_rule.index(rule)
        
        return [current_rule[:this_rule_idx] + v + current_rule[this_rule_idx+1:] for v in this_rule]
            
    return current_rule


rules = {val.split(': ')[0]: [x.split(' ') for x in val.split(': ')[1].replace('"', '').split(' | ')] for val in input_[:input_.index('')]}
messages = input_[input_.index('')+1:]
leaves = rules['0']

final_leaves = set()

while not all(re.match(r"^[ab]*$", ''.join(l)) for l in leaves):
    new_leaves = []
    for leaf in leaves:
        real_leaf = ''.join(leaf)
        if re.match(r"^[ab]*$", real_leaf):
            final_leaves.add(real_leaf)
            continue
            
        toto = transform(leaf, rules)
        new_leaves.extend(toto)
    leaves = deepcopy(new_leaves)

# print(leaves)
# print(final_leaves)

In [110]:
for lll in leaves:
    final_leaves.add(''.join(lll))

In [111]:
counter = 0
for message in messages:
    if message in final_leaves:
        counter += 1
print(counter)

111


In [97]:
new_leaves[0]

['a',
 'b',
 'b',
 'b',
 'b',
 'a',
 'a',
 'a',
 'a',
 'b',
 'b',
 'b',
 'b',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'b',
 'b',
 'a',
 'a',
 'b']

In [99]:
real_leaves = [''.join(t) for t in new_leaves]

In [100]:
real_leaves

['abbbbaaaabbbbaaaaaabbaab', 'abbbbaaaabbbbaaaaaabaaab']

In [24]:
rules

{'0': [['4', '1', '5']],
 '1': [['2', '3'], ['3', '2']],
 '2': [['4', '4'], ['5', '5']],
 '3': [['4', '5'], ['5', '4']],
 '4': [['a']],
 '5': [['b']]}

In [11]:
{val.split(': ')[0]: [x.split(' ') for x in val.split(': ')[1].replace('"', '').split(' | ')] for val in input_[:input_.index('')]}


{'132': [['80', '20'], ['46', '54']],
 '107': [['58', '20'], ['3', '54']],
 '55': [['126', '20'], ['107', '54']],
 '93': [['110', '29']],
 '8': [['42']],
 '75': [['92', '20'], ['103', '54']],
 '113': [['91', '20'], ['35', '54']],
 '94': [['54', '64'], ['20', '21']],
 '46': [['54', '20'], ['110', '54']],
 '26': [['54', '74'], ['20', '94']],
 '61': [['20', '76'], ['54', '25']],
 '128': [['38', '20'], ['29', '54']],
 '129': [['20', '81'], ['54', '49']],
 '21': [['20', '49'], ['54', '9']],
 '103': [['54', '2'], ['20', '51']],
 '9': [['54', '54'], ['20', '20']],
 '82': [['54', '9'], ['20', '29']],
 '127': [['20', '54'], ['54', '54']],
 '59': [['40', '54'], ['87', '20']],
 '0': [['8', '11']],
 '105': [['20', '125'], ['54', '1']],
 '78': [['82', '54'], ['132', '20']],
 '88': [['20', '63'], ['54', '13']],
 '53': [['20', '46'], ['54', '38']],
 '62': [['54', '123'], ['20', '65']],
 '116': [['20', '19'], ['54', '66']],
 '32': [['118', '20'], ['26', '54']],
 '81': [['54', '54']],
 '45': [['54', '5

In [5]:
print(leaves[50:])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [126]:
import re

def transform(string, all_rules):
    new_values = [string]
    
    for rule_name, choices in all_rules.items():
        built_values = []
        for value in new_values:
            for choice in choices:
                built_values.append(value.replace(rule_name, choice))
        
        new_values = built_values
                
    return new_values

rules = {val.split(': ')[0]: val.split(': ')[1].replace(' ', '').replace('"', '').split('|') for val in input_[:input_.index('')]}
messages = input_[input_.index('')+1:]

good_rules = {key: values for key, values in rules.items() if all(re.match(r"^[ab]*$", value) for value in values)}
bad_rules = {key: values for key, values in rules.items() if not all(re.match(r"^[ab]*$", value) for value in values)}

while list(bad_rules.keys()) != []:
#     print(good_rules)
#     print(bad_rules)
    new_rules = {k: [item for sublist in [transform(v, good_rules) for v in vs] for item in sublist] for k, vs in bad_rules.items()}
    good_rules = {**good_rules, **{key: values for key, values in new_rules.items() if all(re.match(r"^[ab]*$", value) for value in values)}}
    bad_rules = {key: values for key, values in new_rules.items() if not all(re.match(r"^[ab]*$", value) for value in values)}
    
    
    #     for key, values in remaining_rules.items():
        
#         print(values, converted_rules)
#         print(list(map(lambda v: v.replace(rule, converted_rules[rule][0]), values)))
        
        
#     print([string.replace(key, value) for string in values for key, value in converted_rules.items()])
        
# for name, values in rules.items():
#     for value in values:
#         value.

# converted_rules
# remaining_rules
# available_rules

best_rules = good_rules['0']

good_messages = 0
for message in messages:
    if message in best_rules:
        good_messages += 1

print(good_messages)

2


In [124]:
bad_rules

{'132': ['46a', 'abb', 'bab'],
 '107': ['58b', '3a'],
 '55': ['107a', 'bb6b'],
 '93': ['b2aa', 'a2aa', 'a2bb', 'b2bb'],
 '8': ['42'],
 '75': ['103a', 'aa2b', 'bb2b'],
 '113': ['bb1b', 'aa1b', '35a'],
 '94': ['b21', 'a64'],
 '26': ['baa4', 'bbb4', 'a74'],
 '61': ['a25', 'b76'],
 '128': ['38b', '2aaa', '2bba'],
 '129': ['a4aa', 'baa', 'a4bb'],
 '21': ['b4bb', 'abb', 'aaa', 'b4aa'],
 '103': ['b51', 'a2'],
 '82': ['b2aa', 'abb', 'aaa', 'b2bb'],
 '59': ['40a', '87b'],
 '0': ['aa1'],
 '105': ['a1', 'bbb5'],
 '78': ['aa2b', 'ab2b', '82a'],
 '88': ['b63', 'aaa', 'aab'],
 '53': ['b46', 'a38'],
 '62': ['abb3', 'b65'],
 '116': ['a66', 'b1aa', 'b1bb'],
 '32': ['26a', '118b'],
 '45': ['a50', 'baa8', 'bbb8'],
 '15': ['73b', 'bb4a'],
 '118': ['5bba', '85b', '5aaa'],
 '64': ['4bbb', '4aab', 'aaa', 'aba'],
 '10': ['baa5', 'abb8', 'bbb5'],
 '11': ['4231'],
 '97': ['b46'],
 '108': ['a88', 'b77'],
 '125': ['2bba', '2aaa', 'bbb'],
 '79': ['b2aa', 'a4bb', 'a4aa', 'b2bb'],
 '43': ['b6', 'a104'],
 '66': ['115

In [74]:
# List of strings 
l = ['sat', 'bat', 'cat', 'mat'] 
  
# map() can listify the list of strings individually 
test = list(map(list, l)) 
print(test) 

[['s', 'a', 't'], ['b', 'a', 't'], ['c', 'a', 't'], ['m', 'a', 't']]


In [ ]:
test_list = ['4', 'kg', 'butter', 'for', '40', 'bucks'] 

# using list comprehension + replace() 
# Replace substring in list of strings 
res = [sub.replace('4', '1') for sub in test_list] 

# Day 21

In [103]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE_2')

response = requests.get(f'https://adventofcode.com/2020/day/21/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['vsnt bdxflb vljjtr ddtbfc kqvpgdhf dhmrf nbmtklt sqlqzh dknpsz cmzsnxn xkmv lgdfltvn gcmcg vltq zvx bjsmg cbsp lp lpfrt vgkq fmmthkn rpgbn dzcsh zjfsc hkjjt tjczdt lfqd ddjbr xjkx hgxjcl vttzcj dmfrzs xjqv ptlf jpx szj qfcsfg zvbmzc mqvk bmfl cgvzlqx tgrx rnlscmt frvr jdcc hkflr ctmcqjf hfq hlqz dpmdps cgjd ttrmcgd bfrq lpvss slvx xmgprh qdgx fdrsmkkq nrpgkq nplshs njcck mhtzd krfnsd kvttspgs srxphcm dmp snmxl htrm hjhvx fxqzh mzjnpdv lffvh vfzrhg cdmxp qbtdklh rcdjqss rm (contains shellfish)',
 'cqtt nplshs cghn xtxzvx txbg vkdfkzh qfcsfg bd lgdfltvn ktrpqv lgvlk vttzcj mzjnpdv srxphcm ttgvz xkmv ssg zvx lfqd mqvk bfrq fgnjv hsxvvn qbtdklh rcdjqss xmgprh slvx cdmxp dqshc jpx rjddcn htrm lpvss vltq zlmxj hkflr tvbtx sxmph pr tfssqq bgfzbxd jfpmgbdn nbmtklt tvqvrb snmxl ssr cmzsnxn hfq cgthc rnnqzt dmp mrgqf dhjvbzc cthbr dxbtm vgkq (contains peanuts)',
 'zvx rpgbn gzpcm fmmthkn hjhvx xjqv srxphcm xzbphmgb hkflr mzjnpdv vttzcj zvbmzc mjsc lnznjm sjsxqxz hndj snmxl bfrq mhtzd hsxvvn rz

In [3]:
input_[0]

'jnlgn xqth jtjtrd vhdt rlsr jjhfbn vvrhc hbrplx fntg dtkr fnlqrk vvbqtz prkblp ffj zkkz vmb xlvrggj qjg sfghbs srjv vphhllt fvjkp qszsz zhszc rzbncvnc mfbfsz xdjk plxs xgc gfzmpxq rmpfkv tglj rsmstv lfckh rkllz qczmsk xpbxbv zdpxfl zfsbvl mmdcl nzqx gzsdm mqv cmffmd gjlql htmb fqrm prtvmb smbdlcg zgzt zcvs qcksk trpbj jbslsbv lhtpf jgjrj tqlbk (contains shellfish)'

In [104]:
import re

ingredients = []
allergens = []

for alpha in input_:
#     alpha = input_[0]
    indices = re.search(r'\(.*\)$', alpha).span()
    ingredients.append(alpha[:indices[0] - 1].split(' '))
    allergens.append(alpha[indices[0] + 10:-1].split(', '))

unique_allergens = set([item for sublist in allergens for item in sublist])
unique_ingredients = set([item for sublist in ingredients for item in sublist])

In [105]:
unique_ingredients


{'bbjfnr',
 'bch',
 'bd',
 'bdxflb',
 'bffb',
 'bfrq',
 'bgfzbxd',
 'bjhtz',
 'bjsmg',
 'bjtrq',
 'bkmrq',
 'blrk',
 'bmfl',
 'brfzm',
 'bsgnt',
 'bxpjj',
 'bxzbk',
 'bzjsxg',
 'cbsp',
 'cbznj',
 'ccnvx',
 'cdmxp',
 'cghn',
 'cgjd',
 'cglpr',
 'cgplh',
 'cgthc',
 'cgvzlqx',
 'clqmstt',
 'cmzsnxn',
 'cqtt',
 'cthbr',
 'ctmcqjf',
 'czt',
 'ddjbr',
 'ddtbfc',
 'ddxjjp',
 'dhjvbzc',
 'dhmrf',
 'dknpsz',
 'dmfrzs',
 'dmp',
 'dnhm',
 'dpmdps',
 'dqshc',
 'dqxcvg',
 'dsshnl',
 'dxbtm',
 'dzcsh',
 'fdrsmkkq',
 'fgnjv',
 'fgp',
 'fkctmn',
 'fkm',
 'fmmthkn',
 'fnlq',
 'frvr',
 'fvlq',
 'fxqzh',
 'gcmcg',
 'gkgzf',
 'gkndl',
 'gmdp',
 'gmmbtr',
 'gtpzdxx',
 'gvkpnl',
 'gzpcm',
 'hcgr',
 'hfq',
 'hgxjcl',
 'hhkl',
 'hjhvx',
 'hkflr',
 'hkjjt',
 'hlqz',
 'hmnf',
 'hndj',
 'hsxvvn',
 'htrm',
 'jdcc',
 'jfpmgbdn',
 'jnmf',
 'jpx',
 'knfs',
 'kpns',
 'kqvpgdhf',
 'krfnsd',
 'ktrpqv',
 'kvlj',
 'kvttspgs',
 'lcdgsh',
 'lcnppvn',
 'lffvh',
 'lfqd',
 'lgdfltvn',
 'lgvlk',
 'lnznjm',
 'lp',
 'lpfrt',
 'l

In [106]:
allergic_dict = {}
real_allergic_ingredients = set()

for unique_allergen in unique_allergens:
#     print(unique_allergen)
    suspects = unique_ingredients

    for idx, allergen in enumerate(allergens):

        if unique_allergen in allergen:
#             print(idx, allergen)
            suspects = suspects.intersection(set(ingredients[idx]))

    print(suspects)
    allergic_dict[unique_allergen] = suspects
    real_allergic_ingredients = real_allergic_ingredients.union(suspects)

print(real_allergic_ingredients)

{'hkflr', 'srxphcm', 'bd'}
{'bfrq'}
{'srxphcm', 'mqvk'}
{'srxphcm', 'bfrq'}
{'hkflr', 'zvx', 'ctmcqjf', 'mqvk'}
{'zvx', 'mqvk', 'snmxl'}
{'srxphcm', 'snmxl', 'bd'}
{'srxphcm', 'zvx', 'mqvk'}
{'hkflr', 'ctmcqjf', 'mqvk', 'snmxl', 'zvx', 'bd', 'srxphcm', 'bfrq'}


In [107]:
not_allergic_ingredients = unique_ingredients - real_allergic_ingredients

counter = 0

for ingredient in [item for sublist in ingredients for item in sublist]:
    if ingredient in not_allergic_ingredients:
        counter += 1

print(counter)

2542


In [108]:
allergic_dict
# the_list = []
# for al in sorted(allergic_dict):
#     for ing in sorted(allergic_dict[al]):
# #         if ing not in the_list:
#         the_list.append(ing)

# ','.join(the_list)

{'eggs': {'bd', 'hkflr', 'srxphcm'},
 'nuts': {'bfrq'},
 'wheat': {'mqvk', 'srxphcm'},
 'peanuts': {'bfrq', 'srxphcm'},
 'fish': {'ctmcqjf', 'hkflr', 'mqvk', 'zvx'},
 'sesame': {'mqvk', 'snmxl', 'zvx'},
 'soy': {'bd', 'snmxl', 'srxphcm'},
 'shellfish': {'mqvk', 'srxphcm', 'zvx'}}

In [109]:
the_dict = {
    'nuts': 'bfrq',
    'peanuts': 'srxphcm',
    'wheat': 'mqvk',
    'fish': 'ctmcqjf',
    'shellfish': 'zvx',
    'sesame': 'snmxl',
    'soy': 'bd',
    'eggs': 'hkflr',
}

In [110]:
the_list = []
for al in sorted(the_dict):
    the_list.append(the_dict[al])

','.join(the_list)

'hkflr,ctmcqjf,bfrq,srxphcm,snmxl,zvx,bd,mqvk'

# Day 22

In [95]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE_2')

response = requests.get(f'https://adventofcode.com/2020/day/22/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_


['Player 1:',
 '4',
 '25',
 '3',
 '11',
 '2',
 '29',
 '41',
 '23',
 '30',
 '21',
 '50',
 '8',
 '1',
 '24',
 '27',
 '10',
 '42',
 '43',
 '38',
 '15',
 '18',
 '13',
 '32',
 '37',
 '34',
 '',
 'Player 2:',
 '12',
 '6',
 '36',
 '35',
 '40',
 '47',
 '31',
 '9',
 '46',
 '49',
 '19',
 '16',
 '5',
 '26',
 '39',
 '48',
 '7',
 '44',
 '45',
 '20',
 '17',
 '14',
 '33',
 '28',
 '22']

In [96]:
decks = []

for idx, elem in enumerate(input_):
    if elem.startswith('Player'):
        current_deck = []
        continue

    if elem == '':
        decks.append(current_deck)
        continue

    current_deck.append(int(elem))

decks.append(current_deck)

def play_round(l1, l2):
    card1 = l1[0]
    card2 = l2[0]
    
    if card1 > card2:
        l1.extend([card1, card2])
    else:
        l2.extend([card2, card1])
    
    l1.pop(0)
    l2.pop(0)
    return l1, l2


In [97]:
from copy import deepcopy

player1, player2 = deepcopy(decks[0]), deepcopy(decks[1])
print(player1, player2)

while player1 != [] and player2 != []:

    player1, player2 = play_round(player1, player2)

len(player1)


[4, 25, 3, 11, 2, 29, 41, 23, 30, 21, 50, 8, 1, 24, 27, 10, 42, 43, 38, 15, 18, 13, 32, 37, 34] [12, 6, 36, 35, 40, 47, 31, 9, 46, 49, 19, 16, 5, 26, 39, 48, 7, 44, 45, 20, 17, 14, 33, 28, 22]


50

In [98]:
result = 0
size = len(player1)

for idx, value in enumerate(player1):
    result += (size - idx) * value

print(result)

33421


In [99]:
# input_ = """Player 1:
# 9
# 2
# 6
# 3
# 1

# Player 2:
# 5
# 8
# 4
# 7
# 10
# """.splitlines()

decks = []

for idx, elem in enumerate(input_):
    if elem.startswith('Player'):
        current_deck = []
        continue

    if elem == '':
        decks.append(current_deck)
        continue

    current_deck.append(int(elem))

decks.append(current_deck)
decks

[[4,
  25,
  3,
  11,
  2,
  29,
  41,
  23,
  30,
  21,
  50,
  8,
  1,
  24,
  27,
  10,
  42,
  43,
  38,
  15,
  18,
  13,
  32,
  37,
  34],
 [12,
  6,
  36,
  35,
  40,
  47,
  31,
  9,
  46,
  49,
  19,
  16,
  5,
  26,
  39,
  48,
  7,
  44,
  45,
  20,
  17,
  14,
  33,
  28,
  22]]

In [100]:
def get_winner(deck_one, deck_two):
    if deck_two == []:
        return [0, deck_one]
    if deck_one == []:
        return [1, deck_two]
    
    return "ERROR"
    
def play_game(one, two):
    old_rounds = []
    
    while (one != [] and two != []):
        if [one, two] in old_rounds:
            return [0, one]
        
        old_rounds.append(deepcopy([one, two]))
#         print(old_rounds)
        
#         print(one, two)
        one, two = recursive_round(one, two)
#         print(one, two)
#         print()
    
    return get_winner(one, two)
    

def recursive_round(deck1, deck2):
    card1 = deck1[0]
    card2 = deck2[0]
    
    if card1 < len(deck1) and card2 < len(deck2):
        subgame_results = play_game(deepcopy(deck1[1:1+card1]), deepcopy(deck2[1:1+card2]))
        # Retourne le gagnant et son deck
        winner = subgame_results[0]
        if winner == 0:
            deck1.extend([card1, card2])
        elif winner == 1:
            deck2.extend([card2, card1])
    else:
        if card1 > card2:
            deck1.extend([card1, card2])
        else:
            deck2.extend([card2, card1])

    deck1.pop(0)
    deck2.pop(0)
    
    return deepcopy(deck1), deepcopy(deck2)


In [101]:
print(play_game(decks[0], decks[1]))

[0, [18, 5, 47, 38, 49, 21, 34, 1, 45, 33, 32, 23, 39, 36, 27, 16, 2, 6, 43, 40, 42, 24, 10, 9, 35, 20, 11, 3, 41, 17, 29, 13, 28, 25, 50, 26, 37, 7, 46, 22, 15, 8, 44, 31, 14, 12, 48, 19, 30, 4]]


In [102]:
yolo = [18, 5, 47, 38, 49, 21, 34, 1, 45, 33, 32, 23, 39, 36, 27, 16, 2, 6, 43, 40, 42, 24, 10, 9, 35, 20, 11, 3, 41, 17, 29, 13, 28, 25, 50, 26, 37, 7, 46, 22, 15, 8, 44, 31, 14, 12, 48, 19, 30, 4]

result = 0
size = len(yolo)

for idx, value in enumerate(yolo):
    result += (size - idx) * value

print(result)

33651


# Day 23

In [455]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE')

response = requests.get(f'https://adventofcode.com/2020/day/23/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['167248359']

In [456]:
# input_ = ['389125467']
input_ = [int(i) for i in list(input_[0])] 
input_.extend([i for i in range(max(input_)+1, 1000001)])
input_[:20]

[1, 6, 7, 2, 4, 8, 3, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [457]:
class Node:
    def __init__(self, data):
        self.data = data
        self.next = None
    
    def __repr__(self):
        return str(self.data)


class CircularLinkedList:
    def __init__(self, list_=None):
        self.head = None
        self.nodes = {}

        if list_:
            node = Node(data=list_[0])
            self.head = node
            self.nodes[node.data] = node
            
            for elem in list_[1:]:
                node.next = Node(data=elem)
                node = node.next
                self.nodes[elem] = node
            node.next = self.head
        
        self.size = sum([1 for node in self])

    def __iter__(self):
        node = self.head
        while (node.next != self.head):
            yield node
            node = node.next
        yield node

    def __repr__(self):        
        return " -> ".join([str(node) for node in self])

    def play(self):
        current_head = self.head
        size = self.size
        group = [0, current_head.next.data, current_head.next.next.data, current_head.next.next.next.data]

        destination_data = (current_head.data - 1) % (size + 1)
        while destination_data in group:
            destination_data = (destination_data - 1) % (size + 1)
        
        # Find the destination node
        destination_node = self.nodes[destination_data]
        
        # Store the nodes required for the wiring
        start_node = current_head.next
        end_node = current_head.next.next.next
        next_head = current_head.next.next.next.next
        next_destination_node = destination_node.next
        
        # Update the wiring
        destination_node.next = start_node
        end_node.next = next_destination_node
        current_head.next = next_head

        # Move the head to the next element
        self.head = current_head.next


In [458]:
l = CircularLinkedList(input_)
# print(l)
# print(l.distances)

In [459]:
for _ in range(10000000):
    l.play()
#     print(l.distances[1])

# print(l)

In [460]:
l.nodes[1].next.data * l.nodes[1].next.next.data

21986479838

In [358]:
l.play()
print(l)
print(l.distances)
print()

l.play()
print(l)
print(l.distances)
print()

l.play()
print(l)
print(l.distances)
print()

2 -> 8 -> 9 -> 1 -> 5 -> 4 -> 6 -> 7 -> 3
{3: 8, 2: 0, 5: 4, 4: 5, 6: 6, 7: 7, 1: 3, 9: 2, 8: 1}

5 -> 4 -> 6 -> 7 -> 8 -> 9 -> 1 -> 3 -> 2
{3: 7, 2: 8, 5: 0, 4: 1, 6: 2, 7: 3, 1: 6, 9: 5, 8: 4}

8 -> 9 -> 1 -> 3 -> 4 -> 6 -> 7 -> 2 -> 5
{3: 3, 2: 7, 5: 8, 1: 2, 9: 1, 8: 0, 7: 6, 6: 5, 4: 4}



# Day 24

In [79]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE')

response = requests.get(f'https://adventofcode.com/2020/day/24/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['wenwwsenwwwwnwwnwwwnwsewseewe',
 'nenenwnenenenwnwnenenwneneseseswnenwwne',
 'newewwwwwwswwwww',
 'esenwwwswwnwnwswnwnwnwewnwsenwswwnwe',
 'esesewseeneswnenwneeewnwneswsenwnee',
 'neseswwswwseseseseseseswnwesesesesesese',
 'sewwswsweswwwswswnwenwswswenesww',
 'wenenesenenewnenenenwenweneneenewse',
 'swswsenwneeseswnwswseseswswswneswsesesesw',
 'wswswswsweswseswnwswswnwswswnwseswswswsee',
 'nesweeeneeenwneswneswewneneenw',
 'swseswswswswswswswswswnwswswswsw',
 'eneneenwneeeeeeneeeese',
 'newenwsenewnesenesenwnenenesenwnwnwne',
 'nwswnenwswneneswenenwneswseeeneneneneee',
 'nenenenwsesenwnwwnwnwsesenwseneswnenenw',
 'eeeseseseeeeeewnwsweseneesesene',
 'wesenwswwwwnewwnwenwnwswwswnww',
 'esenweeeeneeswewenwneweseeee',
 'swswnenenenenenenwneneswneswneneneenwe',
 'neswseenwesenwneswnesewneswnewenwne',
 'nwweswseseswewneeenweeseneeesese',
 'wnwnwwnwnwswswnwswwneeewnwnwsenwnwnw',
 'nenesenewnwneeenene',
 'eseeeewneeeseswnwnwwwseswnewese',
 'neswnenwnwneswenwenwswsewnwneenwwnw',
 'sewswnwswsw

In [80]:
def extract_directions(string):
    list_ = list(string)
    idx = 0
    result = []

    while idx < len(list_):
        if list_[idx] in ['n', 's']:
            result.append(list_[idx] + list_[idx+1])
            idx += 2
        else:
            result.append(list_[idx])
            idx += 1

    return result

def hex_neighbors(tile):
    x, y, z = tile

    return [
        (x+1, y-1, z),
        (x, y-1, z+1),
        (x-1, y, z+1),
        (x-1, y+1, z),
        (x, y+1, z-1),
        (x+1, y, z-1)
    ]

black_tiles = set()

# https://stackoverflow.com/a/16874550/12206696
for v in input_:
    directions = extract_directions(v)
    x, y, z = 0, 0, 0
    
    for direction in directions:
        if direction == 'e':
            x += 1
            y -= 1
        elif direction == 'se':
            y -= 1
            z += 1
        elif direction == 'sw':
            x -= 1
            z += 1
        elif direction == 'w':
            x -= 1
            y += 1
        elif direction == 'nw':
            y += 1
            z -= 1
        elif direction == 'ne':
            x += 1
            z -= 1
    
    coordinates = (x, y, z)
    if coordinates in black_tiles:
        black_tiles.remove(coordinates)
    else:
        black_tiles.add(coordinates)
    
#     print(hex_neighbors(coordinates))

part_one = len(black_tiles)
print(part_one)

269


In [82]:
from copy import deepcopy

for _ in range(100):

    new_black_tiles = set()
    current_white_tiles = {}

    for black_tile in black_tiles:
        black_neighbor_counter = 0

        for neighbor in hex_neighbors(black_tile):
            if neighbor in black_tiles:
                black_neighbor_counter += 1

            if neighbor in current_white_tiles.keys():
                current_white_tiles[neighbor] += 1
            else:
                current_white_tiles[neighbor] = 1

        if black_neighbor_counter not in [0, 3, 4, 5, 6]:
            new_black_tiles.add(black_tile)

    for tile, black_neighbors in current_white_tiles.items():
        if black_neighbors == 2:
            new_black_tiles.add(tile)

    black_tiles = new_black_tiles

print(len(black_tiles))

3667


# Day 25

In [84]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE')

response = requests.get(f'https://adventofcode.com/2020/day/25/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['5099500', '7648211']

In [100]:
def transform(number, loop_size):
    value = 1
    for _ in range(loop_size):
        value = value * number
        value = value % 20201227
    
    return value


def transform_v2(value, number):
    value = value * number
    value = value % 20201227
    
    return value

In [109]:
value = 1
i = 0

while value != 5099500:
    i += 1
    value = transform_v2(value, 7)
    
print(i)

14665099


In [110]:
transform(7648211, 14665099)

11288669

In [99]:
key = None
loop_size = 0

while key != 5099500:
    loop_size += 1
    key = transform(7, loop_size)

print(loop_size)

41592


In [85]:
card_pubkey = int(input_[0])
door_pubkey = int(input_[1])

card_pubkey, door_pubkey

(5099500, 7648211)

# Day 20

In [1]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
SESSION_COOKIE = os.getenv('SESSION_COOKIE')

response = requests.get(f'https://adventofcode.com/2020/day/20/input', cookies={'session': SESSION_COOKIE})
input_ = response.content.decode('UTF-8').splitlines()
input_

['Tile 1171:',
 '.##...#...',
 '.........#',
 '....##....',
 '#.#...##.#',
 '.....#....',
 '.#...#...#',
 '###.......',
 '.#........',
 '#........#',
 '#..##.##.#',
 '',
 'Tile 3331:',
 '..##.#....',
 '#....#...#',
 '#.........',
 '#...#..##.',
 '#.#...#..#',
 '..#...##.#',
 '.......#.#',
 '.....#.###',
 '##..##....',
 '.#.#.###..',
 '',
 'Tile 2273:',
 '.###.#....',
 '........##',
 '#..##.....',
 '...#..#.##',
 '#.........',
 '..#.......',
 '.........#',
 '##.......#',
 '.##..#...#',
 '.##.#.#.#.',
 '',
 'Tile 2179:',
 '.#..#.#...',
 '##.#.#..#.',
 '#...##.#.#',
 '#.........',
 '#..#...#.#',
 '###.##...#',
 '...##....#',
 '.......#..',
 '#.##.....#',
 '#.#####.#.',
 '',
 'Tile 2579:',
 '.#...#....',
 '##.....###',
 '#######..#',
 '#.......#.',
 '###.....##',
 '..#..#....',
 '..#......#',
 '.#...#...#',
 '#.......##',
 '#.####..##',
 '',
 'Tile 1153:',
 '.##.#.....',
 '#.......#.',
 '#...###.#.',
 '..#....#.#',
 '#...#...#.',
 '#...##.###',
 '#..#....#.',
 '#.#......#',
 '#..#.#...#',


In [28]:
input_ = '''Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...'''.splitlines()
input_

['Tile 2311:',
 '..##.#..#.',
 '##..#.....',
 '#...##..#.',
 '####.#...#',
 '##.##.###.',
 '##...#.###',
 '.#.#.#..##',
 '..#....#..',
 '###...#.#.',
 '..###..###',
 '',
 'Tile 1951:',
 '#.##...##.',
 '#.####...#',
 '.....#..##',
 '#...######',
 '.##.#....#',
 '.###.#####',
 '###.##.##.',
 '.###....#.',
 '..#.#..#.#',
 '#...##.#..',
 '',
 'Tile 1171:',
 '####...##.',
 '#..##.#..#',
 '##.#..#.#.',
 '.###.####.',
 '..###.####',
 '.##....##.',
 '.#...####.',
 '#.##.####.',
 '####..#...',
 '.....##...',
 '',
 'Tile 1427:',
 '###.##.#..',
 '.#..#.##..',
 '.#.##.#..#',
 '#.#.#.##.#',
 '....#...##',
 '...##..##.',
 '...#.#####',
 '.#.####.#.',
 '..#..###.#',
 '..##.#..#.',
 '',
 'Tile 1489:',
 '##.#.#....',
 '..##...#..',
 '.##..##...',
 '..#...#...',
 '#####...#.',
 '#..#.#.#.#',
 '...#.#.#..',
 '##.#...##.',
 '..##.##.##',
 '###.##.#..',
 '',
 'Tile 2473:',
 '#....####.',
 '#..#.##...',
 '#.##..#...',
 '######.#.#',
 '.#...#.#.#',
 '.#########',
 '.###.#..#.',
 '########.#',
 '##...##.#.',


In [41]:
import re
from itertools import chain

tiles = {}
# borders = {}

borders = []

for idx, elem in enumerate(input_):
    if elem.startswith('Tile'):
        current_tile = []
        current_name = int(re.search(r"\d+", elem)[0])
        tiles[current_name] = []
        continue

    if elem == '':
        top = tiles[current_name][0]
        bottom = tiles[current_name][-1]
        left = ''
        right = ''
        for line in tiles[current_name]:
            left += line[0]
            right += line[-1]
            
        borders.append((current_name, 'T', top))
        borders.append((current_name, 'B', bottom))
        borders.append((current_name, 'L', left))
        borders.append((current_name, 'R', right))
        borders.append((current_name, 'iT', top[::-1]))
        borders.append((current_name, 'iB', bottom[::-1]))
        borders.append((current_name, 'iL', left[::-1]))
        borders.append((current_name, 'iR', right[::-1]))
        continue

    tiles[current_name].append(elem)

top = tiles[current_name][0]
bottom = tiles[current_name][-1]
left = ''
right = ''
for line in tiles[current_name]:
    left += line[0]
    right += line[-1]

borders.append((current_name, 'T', top))
borders.append((current_name, 'B', bottom))
borders.append((current_name, 'L', left))
borders.append((current_name, 'R', right))
borders.append((current_name, 'iT', top[::-1]))
borders.append((current_name, 'iB', bottom[::-1]))
borders.append((current_name, 'iL', left[::-1]))
borders.append((current_name, 'iR', right[::-1]))

print(len(tiles.keys()))
edges = []
# edge_names = []

for idx, border in enumerate(borders):
    
    compare_borders = [val for val in borders[idx+1:] if val[0] != border[0]]
    
    for compare_border in compare_borders:
        if compare_border[2] == border[2]:
            edge_name = f"{border[0]}-{compare_border[0]}"
            if edge_name not in [val[0] for val in edges]:
                edges.append((edge_name, border[0], compare_border[0], border[1], compare_border[1]))
#     break
    
edges 
# for key, value in borders.items():
    


# ()

    
# part_one = 1
# all_borders = list(chain.from_iterable([v for v in borders.values()]))

# for tile, borders in borders.items():
#     borders_count = 0

#     for border in borders:
#         if all_borders.count(border) > 1:
#             borders_count += 1

#     if borders_count == 4:
#         part_one *= int(tile)

# return part_one

9


[('2311-1427', 2311, 1427, 'T', 'B'),
 ('2311-1951', 2311, 1951, 'L', 'R'),
 ('2311-3079', 2311, 3079, 'R', 'iL'),
 ('1951-2729', 1951, 2729, 'T', 'B'),
 ('1171-2473', 1171, 2473, 'T', 'L'),
 ('1171-1489', 1171, 1489, 'R', 'iR'),
 ('1427-1489', 1427, 1489, 'T', 'B'),
 ('1427-2729', 1427, 2729, 'L', 'R'),
 ('1427-2473', 1427, 2473, 'R', 'B'),
 ('1489-2971', 1489, 2971, 'L', 'R'),
 ('2473-3079', 2473, 3079, 'R', 'iB'),
 ('2971-2729', 2971, 2729, 'B', 'T')]

In [50]:
# l = [edge[1] for edge in edges] + [edge[2] for edge in edges]

# [(val, l.count(val)) for val in set(l) if l.count(val) == 2]


[(3079, 2), (1171, 2), (2971, 2), (1951, 2)]

In [63]:
image = [[0]]

for tile in tiles.keys():
    close_nodes = [edge for edge in edges if edge[1] == tile or edge[2] == tile]
#     print(close_nodes)
    
    if image == [[0]]:
        image[0][0] = tile
    
    for close_node in close_nodes:
        print(close_node)
    
    print()
    
#     break
print(image)
    
    
        
#     tile_edges = 
#     print(tile)

('2311-1427', 2311, 1427, 'T', 'B')
('2311-1951', 2311, 1951, 'L', 'R')
('2311-3079', 2311, 3079, 'R', 'iL')

('2311-1951', 2311, 1951, 'L', 'R')
('1951-2729', 1951, 2729, 'T', 'B')

('1171-2473', 1171, 2473, 'T', 'L')
('1171-1489', 1171, 1489, 'R', 'iR')

('2311-1427', 2311, 1427, 'T', 'B')
('1427-1489', 1427, 1489, 'T', 'B')
('1427-2729', 1427, 2729, 'L', 'R')
('1427-2473', 1427, 2473, 'R', 'B')

('1171-1489', 1171, 1489, 'R', 'iR')
('1427-1489', 1427, 1489, 'T', 'B')
('1489-2971', 1489, 2971, 'L', 'R')

('1171-2473', 1171, 2473, 'T', 'L')
('1427-2473', 1427, 2473, 'R', 'B')
('2473-3079', 2473, 3079, 'R', 'iB')

('1489-2971', 1489, 2971, 'L', 'R')
('2971-2729', 2971, 2729, 'B', 'T')

('1951-2729', 1951, 2729, 'T', 'B')
('1427-2729', 1427, 2729, 'L', 'R')
('2971-2729', 2971, 2729, 'B', 'T')

('2311-3079', 2311, 3079, 'R', 'iL')
('2473-3079', 2473, 3079, 'R', 'iB')

[[2311]]


In [42]:
print('\n'.join(tiles[3079]))
tiles[1427]

#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...


['###.##.#..',
 '.#..#.##..',
 '.#.##.#..#',
 '#.#.#.##.#',
 '....#...##',
 '...##..##.',
 '...#.#####',
 '.#.####.#.',
 '..#..###.#',
 '..##.#..#.']

In [43]:
tiles[2311]

['..##.#..#.',
 '##..#.....',
 '#...##..#.',
 '####.#...#',
 '##.##.###.',
 '##...#.###',
 '.#.#.#..##',
 '..#....#..',
 '###...#.#.',
 '..###..###']

In [45]:
tiles[1951]

['#.##...##.',
 '#.####...#',
 '.....#..##',
 '#...######',
 '.##.#....#',
 '.###.#####',
 '###.##.##.',
 '.###....#.',
 '..#.#..#.#',
 '#...##.#..']

In [47]:
l = []

for idx, val in enumerate(tiles[2311]):
    l.append(tiles[1951][idx] + val)

l

['#.##...##...##.#..#.',
 '#.####...###..#.....',
 '.....#..###...##..#.',
 '#...##########.#...#',
 '.##.#....###.##.###.',
 '.###.#######...#.###',
 '###.##.##..#.#.#..##',
 '.###....#...#....#..',
 '..#.#..#.####...#.#.',
 '#...##.#....###..###']

In [44]:
tiles[1427] + tiles[2311]

['###.##.#..',
 '.#..#.##..',
 '.#.##.#..#',
 '#.#.#.##.#',
 '....#...##',
 '...##..##.',
 '...#.#####',
 '.#.####.#.',
 '..#..###.#',
 '..##.#..#.',
 '..##.#..#.',
 '##..#.....',
 '#...##..#.',
 '####.#...#',
 '##.##.###.',
 '##...#.###',
 '.#.#.#..##',
 '..#....#..',
 '###...#.#.',
 '..###..###']

# day 20

In [ ]:
import re
from itertools import chain

def result(input_):
    tiles = {}
    borders = {}

    for idx, elem in enumerate(input_):
        if elem.startswith('Tile'):
            current_tile = []
            current_name = re.search(r"\d+", elem)[0]
            tiles[current_name] = []
            continue

        if elem == '':
            top = tiles[current_name][0]
            bottom = tiles[current_name][-1]
            left = ''
            right = ''
            for line in tiles[current_name]:
                left += line[0]
                right += line[-1]

            borders[current_name] = [top, bottom, left, right, top[::-1], bottom[::-1], left[::-1], right[::-1]]
            continue

        tiles[current_name].append(elem)

    part_one = 1
    all_borders = list(chain.from_iterable([v for v in borders.values()]))

    for tile, borders in borders.items():
        borders_count = 0

        for border in borders:
            if all_borders.count(border) > 1:
                borders_count += 1

        if borders_count == 4:
            part_one *= int(tile)

    return part_one